<a href="https://colab.research.google.com/gist/allanaasilva/75a2975399fcd8ad819289eba1b161bc/tp1_allana_alves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tema:** Mineração de Padrões Frequentes
### **Base de dados:** https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries


In [ ]:
#Bibliotecas usadas no projeto
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats
from scipy.stats import mannwhitneyu
from mlxtend.preprocessing import TransactionEncoder

import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(precision=5)

%pip install mlxtend --upgrade
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
from mlxtend.frequent_patterns import association_rules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**2. Data understanding (Entendimento dos dados)**


In [ ]:
#Carregando a base de dados
file = ('sample_data/ds_salaries.csv')  
ds_salaries = pd.read_csv(file)
ds_salaries.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [ ]:
ds_salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [ ]:
len(ds_salaries['job_title'].unique())

50

In [ ]:
#Ranking das profissões mais frequentes em Data Science 
count=ds_salaries['job_title'].value_counts().head(10)
fig=px.bar(count,x=count.index,y=count.values,color=count.index,text=count.values,labels={'index':'Profissões','y':'Total','text':'Total'},template='seaborn',title='<b> As 10 profissões mais recorrentes em Data Science')
fig.show()

In [ ]:
#Maiores salários 
fig=px.bar(ds_salaries.groupby('job_title',as_index=False)['salary_in_usd'].max().sort_values(by='salary_in_usd',ascending=False).head(10),x='job_title',y='salary_in_usd',color='job_title',labels={'job_title':'Profissão','salary_in_usd':'Salário'},template='ggplot2',text='salary_in_usd',title='<b> Os 10 maiores salários em Data Science')
fig.show()

In [ ]:
#Maiores salários em média por ocupação
mean_salaries=ds_salaries.groupby('job_title',as_index=False)['salary_in_usd'].mean().sort_values(by='salary_in_usd',ascending=False)
mean_salaries['salary_in_usd']=round(mean_salaries['salary_in_usd'],2)
fig=px.bar(mean_salaries.head(10),x='job_title',y='salary_in_usd',color='job_title',labels={'job_title':'Profissão','salary_in_usd':'Salário Médio'},text='salary_in_usd',template='seaborn',title='<b> Os 10 maiores salários médios em Data Science')
fig.update_traces(textfont_size=8)
fig.show()

In [ ]:
#Distribuição do salário por experiência
px.box(ds_salaries,x='experience_level',y='salary_in_usd',color='experience_level',template='ggplot2',labels={'experience_level':'Nível de Experiência','salary_in_usd':'Salário'},title='<b> Distribuição dos Salários por Experiência')

In [ ]:
#Distribuição dos Salários pelo Tamanho da Empresa
px.box(ds_salaries,x='company_size',y='salary_in_usd',color='company_size',template='ggplot2',labels={'company_size':'Tamanho da Empresa','salary_in_usd':'Salário'},title='<b> Distribuição dos Salários pelo Tamanho da Empresa')

In [ ]:
#Distribuição dos Salários nos Diferentes Anos
px.box(ds_salaries,x='work_year',y='salary_in_usd',color='work_year',template='ggplot2',labels={'work_year':'Ano','salary_in_usd':'Salário'},title='<b> Distribuição dos Salários nos Diferentes Anos')

Não é do interesse do presente trabalho analisar o crescimento dos salários com o passar do tempo, já que é esperado que isso aconteça, e sim a frequência com que as demais variáveis aparecem nos grupos de salários, dos mais baixos aos mais altos. Por isso será feita uma comparação das amostras dos grupos acima.

In [ ]:
print(stats.shapiro(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2020]))
print(stats.shapiro(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2021]))
print(stats.shapiro(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2022]))

ShapiroResult(statistic=0.7731199264526367, pvalue=3.466301690124851e-09)
ShapiroResult(statistic=0.8254565000534058, pvalue=7.066709481582008e-15)
ShapiroResult(statistic=0.9619481563568115, pvalue=2.2350339179411094e-07)


In [ ]:
print(mannwhitneyu(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2020], ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2021],))
print(mannwhitneyu(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2020], ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2022],))
print(mannwhitneyu(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2021], ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2022],))

MannwhitneyuResult(statistic=7362.0, pvalue=0.4644467986780825)
MannwhitneyuResult(statistic=7126.0, pvalue=5.630618734848829e-07)
MannwhitneyuResult(statistic=24196.0, pvalue=4.338794861906176e-09)


Como rejeitamos a hipótese de que as amostras dos anos 2020 e 2021 vêm de uma mesma distribuição da amostra de 2022, iremos trabalhar apenas com os salários de 2022 e eliminar a covariável work_year (na etapa de Data Preparation).

In [ ]:
#Distribuição dos Salários
px.histogram(ds_salaries,x=ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2022],marginal='rug',template='seaborn',title='<b> Distribuição dos Salários em 2022')

In [ ]:
#Nos ajudará a definir as faixas para os salários em Data Preparation
(ds_salaries['salary_in_usd'].loc[ds_salaries['work_year'] == 2022]).describe()

count       318.000000
mean     124522.006289
std       58183.647585
min       10000.000000
25%       81666.000000
50%      120000.000000
75%      160000.000000
max      405000.000000
Name: salary_in_usd, dtype: float64

In [ ]:
bad=np.quantile(ds_salaries['salary_in_usd'], 0.33)
good=np.quantile(ds_salaries['salary_in_usd'], 0.66)
print(bad,good)

76940.0 130000.0


In [ ]:
crosstab = pd.crosstab(ds_salaries['employee_residence'], ds_salaries['company_location'])
stats.chi2_contingency(crosstab)

(23187.380792483484,
 0.0,
 2744,
 array([[1.48270e-02, 4.94234e-03, 1.97694e-02, ..., 4.94234e-03,
         1.75453e+00, 4.94234e-03],
        [4.94234e-03, 1.64745e-03, 6.58979e-03, ..., 1.64745e-03,
         5.84843e-01, 1.64745e-03],
        [1.48270e-02, 4.94234e-03, 1.97694e-02, ..., 4.94234e-03,
         1.75453e+00, 4.94234e-03],
        ...,
        [4.94234e-03, 1.64745e-03, 6.58979e-03, ..., 1.64745e-03,
         5.84843e-01, 1.64745e-03],
        [1.64086e+00, 5.46952e-01, 2.18781e+00, ..., 5.46952e-01,
         1.94168e+02, 5.46952e-01],
        [1.48270e-02, 4.94234e-03, 1.97694e-02, ..., 4.94234e-03,
         1.75453e+00, 4.94234e-03]]))

Como existe uma relação entre a residência do empregado com o local da empresa onde ele trabalha, levaremos em conta para o modelo apenas a variável 'company_location'.

In [ ]:
ds_salaries.value_counts('employment_type')

employment_type
FT    588
PT     10
CT      5
FL      4
dtype: int64

Como a grande maioria dos trabalhadores trabalham como Full Time, a variável 'employment_type' não será levada em conta.

##**3. Data preparation (Preparação dos dados)**

In [ ]:
#Apenas dados do ano de 2022
ds_salaries_2022=ds_salaries.loc[ds_salaries['work_year'] == 2022]

In [ ]:
#Categorização da variável 'salary_in_usd' de acordo com as informações encontradas em Data Understanding 
ds_salaries_2022['salary'] = 0
ds_salaries_2022.loc[ds_salaries_2022['salary_in_usd'] <= 80000, 'salary'] = 'bad'
ds_salaries_2022.loc[((ds_salaries_2022['salary_in_usd'] > 80000) & (ds_salaries_2022['salary_in_usd'] <= 130000)), 
'salary'] = 'good'
ds_salaries_2022.loc[ds_salaries_2022['salary_in_usd'] > 130000, 'salary'] = 'very_good'

In [ ]:
#Variáveis que serão utilizadas
ds_salaries_2022=ds_salaries_2022.reset_index()
ds_salaries_2022=ds_salaries_2022[['experience_level','job_title','salary',
'remote_ratio','company_location','company_size']]

In [ ]:
ds_salaries_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   experience_level  318 non-null    object
 1   job_title         318 non-null    object
 2   salary            318 non-null    object
 3   remote_ratio      318 non-null    int64 
 4   company_location  318 non-null    object
 5   company_size      318 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.0+ KB


In [ ]:
#Separação dos datasets
ds_salaries_bad=ds_salaries_2022.loc[ds_salaries_2022['salary']=='bad']
ds_salaries_good=ds_salaries_2022.loc[ds_salaries_2022['salary']=='good']
ds_salaries_very_good=ds_salaries_2022.loc[ds_salaries_2022['salary']=='very_good']

In [ ]:
ds_salaries_bad.info()
ds_salaries_good.info()
ds_salaries_very_good.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 15 to 312
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   experience_level  79 non-null     object
 1   job_title         79 non-null     object
 2   salary            79 non-null     object
 3   remote_ratio      79 non-null     int64 
 4   company_location  79 non-null     object
 5   company_size      79 non-null     object
dtypes: int64(1), object(5)
memory usage: 4.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 2 to 315
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   experience_level  104 non-null    object
 1   job_title         104 non-null    object
 2   salary            104 non-null    object
 3   remote_ratio      104 non-null    int64 
 4   company_location  104 non-null    object
 5   company_size      104 non-null    object
dtypes

##**4. Modeling (Modelagem)**

O algoritmo escolhido para a mineração de padrões frequentes foi o FP Growth. Ele será usado nos quatro datasets separados na etapa anterior e no final será feita a comparação dos resultados obtidos por faixa de salário.

In [ ]:
transactionencoder = TransactionEncoder()
min_support = 0.3
def frequent_itemsets(df,i):
  del df['salary']
  list_features = [] 
  for i in range(0, i):
    list_features.append([str(df.values[i,j]) for j in range(0, 5)]) 
  transactionencoder.fit(list_features)
  encoded_transactions = transactionencoder.transform(list_features)
  df_encoded_transactions = pd.DataFrame(encoded_transactions, columns=transactionencoder.columns_)
  frequent_itemsets = fpgrowth(df_encoded_transactions, min_support=min_support, use_colnames = True)
  return(frequent_itemsets)


In [ ]:
print('#'*30)
print('Bad Salaries:')
frequent_itemsets_b=frequent_itemsets(ds_salaries_bad,79)
print(frequent_itemsets_b)
print('#'*30)

print('Good Salaries:')
frequent_itemsets_g=frequent_itemsets(ds_salaries_good,104)
print(frequent_itemsets_g)
print('#'*30)

print('Very Good Salaries:')
frequent_itemsets_vg=frequent_itemsets(ds_salaries_very_good,135)
print(frequent_itemsets_vg)
print('#'*30)

##############################
Bad Salaries:
    support         itemsets
0  0.696203              (M)
1  0.645570            (100)
2  0.303797  (Data Engineer)
3  0.594937             (MI)
4  0.443038         (M, 100)
5  0.417722          (M, MI)
6  0.354430        (MI, 100)
##############################
Good Salaries:
     support                   itemsets
0   0.884615                        (M)
1   0.759615                       (US)
2   0.682692                      (100)
3   0.644231                       (SE)
4   0.384615             (Data Analyst)
5   0.692308                    (M, US)
6   0.615385                   (M, 100)
7   0.567308                  (US, 100)
8   0.528846               (M, US, 100)
9   0.596154                    (M, SE)
10  0.567308                   (SE, US)
11  0.509615                  (SE, 100)
12  0.538462                (M, SE, US)
13  0.490385               (M, SE, 100)
14  0.471154              (SE, US, 100)
15  0.461538           (M, SE, US, 10

Comentários a respeito dos resultados obtidos no relatório final.

##**5. Evaluation (Validação)**


In [ ]:
from mlxtend.frequent_patterns import association_rules
def evaluation(frequent_itemsets):
  association=pd.DataFrame(association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7))
  return (association)

In [ ]:
print('#'*30)
print('Bad Salaries:')
evaluation_b=evaluation(frequent_itemsets_b)
print(evaluation_b)
print('#'*30)

print('Good Salaries:')
evaluation_g=evaluation(frequent_itemsets_g)
print(evaluation_g)
print('#'*30)

print('Very Good Salaries:')
evaluation_vg=evaluation(frequent_itemsets_vg)
print(evaluation_vg)
print('#'*30)

##############################
Bad Salaries:
  antecedents consequents  antecedent support  consequent support   support  \
0        (MI)         (M)            0.594937            0.696203  0.417722   

   confidence      lift  leverage  conviction  
0    0.702128  1.008511  0.003525    1.019892  
##############################
Good Salaries:
               antecedents   consequents  antecedent support  \
0                      (M)          (US)            0.884615   
1                     (US)           (M)            0.759615   
2                    (100)           (M)            0.682692   
3                     (US)         (100)            0.759615   
4                    (100)          (US)            0.682692   
5                  (M, US)         (100)            0.692308   
6                 (M, 100)          (US)            0.615385   
7                (US, 100)           (M)            0.567308   
8                    (100)       (M, US)            0.682692   
9             

Comentários a respeito dos resultados obtidos no relatório final.